In [1]:
from importlib.metadata import version

print('torch version:', version('torch'))
print('tiktoken version:', version('tiktoken'))
import os

torch version: 2.4.1
tiktoken version: 0.7.0


In [2]:
os.getcwd()

'/Users/yash/Desktop/ATMRI/OneDrive - Nanyang Technological University/ATMRI/LLM_workshop_sebR/02'

### Tokenizing text

In [4]:
with open("the-verdict.txt", "r", encoding='utf-8') as f:
    raw_text = f.read()

print('total characters:', len(raw_text))
print(raw_text[:99])

total characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [5]:
import re 
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
print(preprocessed[:38])

['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius', '--', 'though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough', '--', 'so', ' ', 'it', ' ', 'was', ' ', 'no', ' ']


In [7]:
print('number of tokens', len(preprocessed))
print(len(set(preprocessed)))

number of tokens 9235
1133


### Converting tokens to token IDs

In [10]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1133


In [15]:
vocab = {token:integer for integer, token in enumerate(all_words)}

for i, item in enumerate(vocab.items()):
    print(i, item)
    if i > 10:
        break

0 ('', 0)
1 ('\n', 1)
2 (' ', 2)
3 ('!', 3)
4 ('"', 4)
5 ("'", 5)
6 ('(', 6)
7 (')', 7)
8 (',', 8)
9 ('--', 9)
10 ('.', 10)
11 (':', 11)


In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {1:s for s, i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        
        return text

#### tokenizers such as BytePair encoding are able to encode unknown words as well.

In [16]:
import importlib
import importlib.metadata
import tiktoken 

print('tiktoken version:', importlib.metadata.version('tiktoken'))

tiktoken version: 0.7.0


In [17]:
tokenizer = tiktoken.get_encoding("gpt2")

In [18]:
text = ("Hello, do you liek tea? <|endoftext>| In the sunlit terraces"
        "of someunknownPlace."
        )
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 6486, 74, 8887, 30, 1279, 91, 437, 1659, 5239, 29, 91, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [20]:
strings = tokenizer.decode(integers)
strings

'Hello, do you liek tea? <|endoftext>| In the sunlit terracesof someunknownPlace.'

#### During pre-training, the idea is to move usign strides (just like CNNs), so that the LLM does not see too many repeated texts

### Data sampling with sliding window

In [23]:
from supplementary import create_dataloader_v1

dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
#for models like GPT max_length is usually 1024
data_iter = iter(dataloader)
inputs, Targets = next(data_iter)
print('inputs:\n', inputs)
print('\nTargets:\n', Targets)

inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
